In [28]:
import json
import datetime
import logging
from confluent_kafka import Consumer, Producer

In [49]:
def validate_transaction(event):
    errors = []
    try:
        total_expected = event['amount'] + event['vat_amount'] + event['commission_amount']
        if event['total_amount'] != total_expected:
            errors.append('ERR_AMOUNT')
    except:
        errors.append('ERR_AMOUNT')

    try:
        now = datetime.datetime.utcnow()
        ts = datetime.datetime.fromisoformat(event['timestamp'].replace("Z", ""))
        day_per_sec = 86400
        if ts > now or (now - ts).total_seconds() > day_per_sec:
            errors.append('ERR_TIME')
    except:
        errors.append('ERR_TIME')
        
        
    try:
        now = datetime.datetime.utcnow()
        ts = datetime.datetime.fromisoformat(event['timestamp'].replace("Z", ""))
        day_per_sec = 86400
        if ts > now or (now - ts).total_seconds() > day_per_sec:
            errors.append('ERR_TIME')
    except:
        errors.append('ERR_TIME')


    try:
        if event['payment_method'] == 'mobile':
            os = event.get('device_info', {}).get('os', None)
            if os not in ['Android', 'iOS']:
                errors.append('ERR_DEVICE')
    except:
        errors.append('ERR_DEVICE')

    return errors

In [ ]:
logging.basicConfig(level=logging.INFO)

consumer_conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'darooghe-consumer',
    'auto.offset.reset': 'earliest',
}
consumer = Consumer(consumer_conf)
consumer.subscribe(['darooghe.transactions'])

producer = Producer({'bootstrap.servers': 'localhost:9092'})

try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:
            continue
        if msg.error():
            logging.error(f"Consumer error: {msg.error()}")
            continue

        try:
            event = json.loads(msg.value().decode('utf-8'))
            # raw_event = json.loads(msg.value().decode('utf-8'))
            # event = parse_transaction(raw_event)
            transaction_id = event.get('transaction_id', 'unknown')

            errors = validate_transaction(event)

            if errors:
                error_msg = {
                    'transaction_id': transaction_id,
                    'errors': errors,
                    'original_data': event
                }
                producer.produce(
                    'darooghe.error_logs',
                    key=transaction_id,
                    value=json.dumps(error_msg, default=str)
                )
                logging.warning(f"Invalid transaction: {transaction_id}, errors: {errors}")
            else:
                logging.info(f"Valid transaction: {transaction_id}")

        except Exception as e:
            logging.error(f"Parse error: {e}")

        producer.poll(0)

except KeyboardInterrupt:
    logging.info("Shutting down...")

finally:
    consumer.close()
    producer.flush()

In [ ]:

consumer_conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'darooghe-consumer',
    'auto.offset.reset': 'earliest',
}
consumer = Consumer(consumer_conf)
consumer.subscribe(['darooghe.transactions'])

try:
    msg = consumer.poll(timeout=10.0)
    if msg is None:
        print("No message received.")
    elif msg.error():
        print(f"Error: {msg.error()}")
    else:
        event = json.loads(msg.value().decode('utf-8'))
        # raw_event = json.loads(msg.value().decode('utf-8'))
        # event = parse_transaction(raw_event)
        print("Received message:")
        print(json.dumps(event, indent=2))

        errors = validate_transaction(event)
        if errors:
            print("Validation Errors:", errors)
        else:
            print("Message is VALID!")

finally:
    consumer.close()


In [84]:
def parse_transaction(raw_event):
    try:
        event = {}
        event['transaction_id'] = str(raw_event['transaction_id'])
        event['timestamp'] = datetime.datetime.fromisoformat(raw_event['timestamp'].replace('Z', ''))
        event['customer_id'] = str(raw_event['customer_id'])
        event['merchant_id'] = str(raw_event['merchant_id'])
        event['merchant_category'] = str(raw_event['merchant_category'])
        event['payment_method'] = str(raw_event['payment_method'])
        event['amount'] = int(raw_event['amount'])
        event['location'] = {
            'lat': float(raw_event['location']['lat']),
            'lng': float(raw_event['location']['lng']),
        }
        event['device_info'] = raw_event.get('device_info', {})
        event['status'] = str(raw_event['status'])
        event['commission_type'] = str(raw_event['commission_type'])
        event['commission_amount'] = int(raw_event['commission_amount'])
        event['vat_amount'] = int(raw_event['vat_amount'])
        event['total_amount'] = int(raw_event['total_amount'])
        event['customer_type'] = str(raw_event['customer_type'])
        event['risk_level'] = int(raw_event['risk_level'])
        event['failure_reason'] = raw_event.get('failure_reason', None)
        return event
    except (KeyError, ValueError, TypeError) as e:
        print(f"Schema error: {e}")
        return None
